In [ ]:
print("chinni")

chinni


In [ ]:
## DEBUG ##
# !apt-get update
# !apt-get install -y build-essential cmake rustc

# !pip install -U transformers sentence-transformers

# import os
# os._exit(00)

# import transformers
# print(transformers.__version__) # 4.56.1

# from transformers import Trainer
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('all-MiniLM-L6-v2')


# from sentence_transformers import SentenceTransformer

# try:
#     # Initialize the SentenceTransformer model
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     print("Model loaded successfully!")
# except Exception as e:
#     print(f"Error: {e}")




In [ ]:
!pip install --upgrade pip



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install langchain langchain-core langchain-community
!pip install  pypdf pymupdf tqdm
!pip install sentence-transformers
!pip install faiss-cpu chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s  0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [langchain-community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 109.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymupdf]
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 111.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 138.0 MB/s  0:00

# Basic Python

# Data Ingestion

## Document

In [ ]:
from langchain_core.documents import Document

In [ ]:
doc = Document(
    page_content="Hello, Chinni!, This is your first set-up for RAG, Please do more!!!!",
    metadata=
    {"source": "example.txt",
     "author": "Chinni Krishna",
     "page": 1,
     "status": "active",
     "date_created": "2025-06-06",
     "version": "1.0"},
    additional_info={"notes": "This is a sample document for testing."},
    tags=["sample", "test"]
    )

In [ ]:
doc

Document(metadata={'source': 'example.txt', 'author': 'Chinni Krishna', 'page': 1, 'status': 'active', 'date_created': '2025-06-06', 'version': '1.0'}, page_content='Hello, Chinni!, This is your first set-up for RAG, Please do more!!!!')

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

/content


In [ ]:
os.makedirs('data/text_files', exist_ok=True)

In [ ]:
sample_text = {"data/text_files/abc.txt": "This is my first sample text file", "data/text_files/mno.txt": "This is a sample text file.", "data/text_files/xyz.txt": "This is another sample text file."}

for file_path, content in sample_text.items():
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)
        file.close()


print("Files have been created successfully.")


Files have been created successfully.


# LOADER

## Text Loader

In [ ]:
## Single file Loader

from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader = TextLoader(f'data/text_files/abc.txt', encoding='utf-8')

data1 = loader.load()
print(data1)

[Document(metadata={'source': 'data/text_files/abc.txt'}, page_content='This is my first sample text file')]


In [ ]:
### Multi-file Loader

from langchain.document_loaders import DirectoryLoader
dir_loader = DirectoryLoader(
    'data/text_files',
    glob='*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=True
    )
data2 = dir_loader.load()
print(data2)

100%|██████████| 3/3 [00:00<00:00, 3707.40it/s]

[Document(metadata={'source': 'data/text_files/xyz.txt'}, page_content='This is another sample text file.'), Document(metadata={'source': 'data/text_files/abc.txt'}, page_content='This is my first sample text file'), Document(metadata={'source': 'data/text_files/mno.txt'}, page_content='This is a sample text file.')]


## PDF Loader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

dir_loader = DirectoryLoader(
    'data/pdf',
    glob='**/*.pdf',
    loader_cls=PyMuPDFLoader,
    show_progress=True
    )
pdf_documents = dir_loader.load()
print(pdf_documents)


FileNotFoundError: Directory not found: 'data/pdf'

In [ ]:
type(pdf_documents[0])

langchain_core.documents.base.Document

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")

        except Exception as e:
            print(f"  ✗ Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("data")

Found 1 PDF files to process

Processing: chinni-resume.pdf
  ✓ Loaded 2 pages

Total documents loaded: 2


In [7]:
all_pdf_documents


NameError: name 'all_pdf_documents' is not defined

# Chunks

In [ ]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [1]:
chunks=split_documents(all_pdf_documents)
chunks

NameError: name 'split_documents' is not defined

# Embedding & VectorStoreDB

## Embedding

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Optional, Tuple, Callable, Iterable, Type, Iterator
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


##  VectorStore

In [ ]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [ ]:
len(chunks)

11

In [2]:
chunks

NameError: name 'chunks' is not defined

In [ ]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store in the vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 11 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (11, 384)
Adding 11 documents to vector store...
Successfully added 11 documents to vector store
Total documents in collection: 22


# RAG Retriever Pipeline from VectorStore

In [ ]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [ ]:
rag_retriever

In [3]:
rag_retriever.retrieve("what do you know about company?")


NameError: name 'rag_retriever' is not defined

# RAG Pipeline - VectorDB To LLM Output Generation

In [ ]:
CHATGPT_API_KEY = ""

In [4]:
# TESTING
from openai import OpenAI

client = OpenAI(api_key=CHATGPT_API_KEY)  # Replace with your actual key

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello"}
    ]
)

print(response.choices[0].message.content)


NameError: name 'CHATGPT_API_KEY' is not defined

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from typing import Optional

In [ ]:
class OpenAILLM:
    def __init__(self, model_name: str = "gpt-4", api_key: Optional[str] = None):
        """
        Initialize OpenAI ChatGPT LLM

        Args:
            model_name: OpenAI model name (e.g., "gpt-4", "gpt-3.5-turbo")
            api_key: OpenAI API key (or set OPENAI_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = CHATGPT_API_KEY

        if not self.api_key:
            raise ValueError("OpenAI API key is required. Set OPENAI_API_KEY environment variable or pass api_key parameter.")

        self.llm = ChatOpenAI(
            openai_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )

        print(f"Initialized OpenAI LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context

        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length

        Returns:
            Generated response string
        """
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant for Chinni Krishna. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )

        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting

        Args:
            query: User question
            context: Retrieved context

        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""

        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"


In [ ]:
try:
    openai_llm = OpenAILLM(api_key=CHATGPT_API_KEY)
    print("OPENAI LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your OPENAI KEY environment variable to use the LLM.")
    openai_llm = None

Initialized OpenAI LLM with model: gpt-4
OPENAI LLM initialized successfully!


In [5]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("DevOps")

NameError: name 'rag_retriever' is not defined

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

load_dotenv()

### Initialize the OpenAI LLM (set your OPENAI_API_KEY in environment)
openai_api_key = CHATGPT_API_KEY

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-4",  # or "gpt-3.5-turbo"
    temperature=0.1,
    max_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    # Retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""

    if not context:
        return "No relevant context found to answer the question."

    # Generate the answer using OpenAI LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {query}

Answer:"""

    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content


In [6]:
answer=rag_simple("DevOps",rag_retriever,llm)
print(answer)

NameError: name 'rag_simple' is not defined